In [4]:
!pip install langchain langchain-community langchain-huggingface langchain-text-splitters langchain-chroma \
             pypdf sentence-transformers transformers torch accelerate bitsandbytes --quiet

In [5]:
from huggingface_hub import notebook_login

notebook_login()

In [6]:
import os
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"device: {device}")

EMBEDDING_MODEL_NAME = "PartAI/Tooka-SBERT-V2-Large"

LLM_MODEL_NAME = "PartAI/Dorna2-Llama3.1-8B-Instruct"

PERSIST_DIRECTORY = "./chromadb_persian_rag"
PDF_FILE_NAME = None

device: cuda


In [7]:
from google.colab import files

uploaded = files.upload()

PDF_FILE_NAME = list(uploaded.keys())[0]
print(f"'{PDF_FILE_NAME}' succesfully uploaded!")

Saving gozaresh.pdf to gozaresh.pdf
'gozaresh.pdf' succesfully uploaded!


In [9]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = PyPDFLoader(PDF_FILE_NAME)
documents = loader.load()

print(f"document has {len(documents)} pages!")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)
chunks = text_splitter.split_documents(documents)
print(f"number of chunks: {len(chunks)}")


document has 17 pages!
number of chunks: 228


In [10]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding_model_kwargs = {'device': device.type}
embedding_encode_kwargs = {'normalize_embeddings': True}

embeddings = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    model_kwargs=embedding_model_kwargs,
    encode_kwargs=embedding_encode_kwargs
)
test = embeddings.embed_query("یک جمله تست!")
# len(test)

<ipython-input-10-996686571e64>:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/205 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/6.84k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.97M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/828 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

1024

In [11]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
), model_name='PartAI/Tooka-SBERT-V2-Large', cache_folder=None, model_kwargs={'device': 'cuda'}, encode_kwargs={'normalize_embeddings': True}, multi_process=False, show_progress=False)

In [12]:
from langchain_community.vectorstores import Chroma

vector_store = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory=PERSIST_DIRECTORY
)

In [16]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface.llms import HuggingFacePipeline

tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL_NAME)

model = AutoModelForCausalLM.from_pretrained(
    LLM_MODEL_NAME,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    load_in_4bit=True,
    # bnb_4bit_quant_type="nf4",
    # bnb_4bit_compute_dtype=torch.bfloat16
)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

text_generation_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128,
    do_sample=True,
    temperature=0.2,
    top_p=0.9,
    eos_token_id=terminators
)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Device set to use cuda:0


In [17]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)

# system_prompt_fa = """
# شما یک دستیار هوش مصنوعی بسیار مفید و دقیق هستید. وظیفه شما این است که به سوالات کاربر فقط و فقط بر اساس اطلاعات زمینه (متن‌های بازیابی شده) که در ادامه ارائه می‌شود، پاسخ دهید.
# پاسخ‌های شما باید کاملاً مبتنی بر زمینه باشد. از دانش عمومی خودتان به هیچ وجه استفاده نکنید.
# اگر پاسخ سوال به طور واضح در زمینه موجود نیست، باید به صراحت بیان کنید که "اطلاعات کافی برای پاسخ به این سوال در متون ارائه شده وجود ندارد."
# سعی نکنید پاسخی خارج از اطلاعات زمینه بسازید یا حدس بزنید.
# پاسخ خود را به زبان فارسی روان، واضح و دقیق ارائه دهید. پاسخ باید تا حد امکان کامل و در عین حال مختصر باشد و مستقیماً به سوال کاربر بپردازد.
# """

system_prompt_fa = """
بر اساس اطلاعات زمینه (متن‌های بازیابی شده)، به سوالات کاربر پاسخ بده. اگر پاسخ سوال رو پیدا نکردی، بگو "اطلاعات کافی موجود نیست".
"""

# user_prompt_template_fa = """
# بر اساس اطلاعات زمینه زیر:
# <زمینه>
# {context}
# </زمینه>

# به این سوال کاربر پاسخ دقیق بده:
# <سوال_کاربر>
# {question}
# </سوال_کاربر>

# پاسخ دقیق و مبتنی بر زمینه به زبان فارسی:
# """

# prompt = ChatPromptTemplate.from_messages([
#     SystemMessagePromptTemplate.from_template(system_prompt_fa),
#     HumanMessagePromptTemplate.from_template(user_prompt_template_fa)
# ])


simple_persian_prompt_template = f"""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>{system_prompt_fa}<|eot_id|><|start_header_id|>user<|end_header_id|>
زمینه:
{{context}}

سوال کاربر: {{question}}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
پاسخ دقیق و مبتنی بر زمینه به زبان فارسی:
"""
PROMPT = PromptTemplate(
    template=simple_persian_prompt_template, input_variables=["context", "question"]
)

chain_type_kwargs = {"prompt": PROMPT}

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=False,
    chain_type_kwargs=chain_type_kwargs
)

In [21]:
if qa_chain:
    queries_persian = [
        "داور فینال لیگ قهرمانان اروپا 2026 کی بود؟",
        "دقیقه 67 رو توصیف کن",
        "بارسلونا چند گل زد؟",
        "در مورد لامین یامال بهم بگو",
        "آب و هوا چطور بود؟"
    ]

    for query in queries_persian:
        print(f"query: {query}")
        print("processing ...")
        result = qa_chain.invoke({"query": query})

        print("response:")
        response_text = result["result"].strip()
        print(response_text)


        # print("\n")
        # if result.get("source_documents"):
        #     for i, doc in enumerate(result["source_documents"]):
        #         print(f"resource - {i+1}")
        #         page_content_preview = " ".join(doc.page_content.splitlines())
        #         print(f"page_content_preview: {page_content_preview[:250]}...")
        #         if doc.metadata.get("page"):
        #             print(f"page: {doc.metadata['page'] + 1}")
        #         print("=========================================================")
        # else:
        #     print("not found!")
else:
    print("QA chain not available")

query: داور فینال لیگ قهرمانان اروپا 2026 کی بود؟
processing ...


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


response:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
بر اساس اطلاعات زمینه (متن‌های بازیابی شده)، به سوالات کاربر پاسخ بده. اگر پاسخ سوال رو پیدا نکردی، بگو "اطلاعات کافی موجود نیست". 
<|eot_id|><|start_header_id|>user<|end_header_id|>
زمینه:
گزارش زنده فینال لیگ قهرمانان اروپا۶۲۰۲:  بارسلونا در مقابل منچسترسیتی  -  شبی به

قهرمان لیگ قهرمانان اروپا در سال  ۶۲۰۲  شد! بازیکنان و کادر فنی بارسلونا به داخل زمین هجوم

فینال لیگ قهرمانان اروپا در سال  ۶۲۰۲، یک بازی کلاسیک و به یاد ماندنی بود. بارسلونا با وجود

سال اخیر، به فینال لیگ قهرمانان صعود کنند. ارلینگ هالند، ماشین گلزنی نروژی، در کنار

• مهاجم:  ارلینگ هالند  
سرمربی: پپ گواردیولا  
داور بازی:  آقای کلمن تورپن از فرانسه

سوال کاربر: داور فینال لیگ قهرمانان اروپا 2026 کی بود؟<|eot_id|><|start_header_id|>assistant<|end_header_id|>
پاسخ دقیق و مبتنی بر زمینه به زبان فارسی:
آقای کلمن تورپن، داور فرانسوی، مسئولیت هدایت بازی فینال لیگ قهرمانان اروپا در سال ۲۰۲۶ را بر عهده داشت.
query: دقیقه 67 رو توصیف کن
processing ...
r

In [25]:
!git add .
!git commit -m "add ipynb"
!git push -u origin main

On branch main
Your branch is ahead of 'origin/main' by 1 commit.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean
Enumerating objects: 4, done.
Counting objects: 100% (4/4), done.
Delta compression using up to 2 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 21.18 KiB | 4.24 MiB/s, done.
Total 3 (delta 1), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
remote: error: GH013: Repository rule violations found for refs/heads/main.
remote: 
remote: - GITHUB PUSH PROTECTION
remote:   —————————————————————————————————————————
remote:     Resolve the following violations before pushing again
remote: 
remote:     - Push cannot contain secrets
remote: 
remote:     
remote:      (?) Learn how to resolve a blocked push
remote:      https://docs.github.com/code-security/secret-scanning/working-with-secret-scanning-and-push-protection/working-with-push-protection-from-the-comm